# Libraries

In [ ]:
import math
import numpy as np
import pandas as pd
import gc
import keras
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cv2
import random


from skimage.transform import resize
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras import Model
from tensorflow.keras.callbacks import  ModelCheckpoint
from tensorflow.keras.layers import *
from keras.utils.vis_utils import plot_model
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.activations import sigmoid, tanh
import random
from tensorflow.keras.metrics import *
from tensorflow.keras.layers import *


from PIL import Image
from statistics import mean


# Custom modules

from metrics import *
from baseline_classifier_models import *
from baseline_segmentation_models import *

# Set Parameters

In [ ]:
img_dims= 256
batch_= 8
seed_ = 3+21+35+40+63
class_type = 'categorical' #binary/categorical
task_ = 'mask' # 'mask' : segementation, 'recon' : reconstruction

train_segmentation_only = 0
train_classification_only = 0
train_combined = 1

lr_ = 1e-4

# Baseline Classification Models

In [ ]:
model = EfficientNetClassifier(img_dims, class_type, type_="B4", truncation=False) # B0-B7

# model = DenseNetClassifier(img_dims, class_type, type_="121", truncation=False) # 121, 169, 201
# model = ResNetClassifier(img_dims, class_type, type_="50") # 50, 101, 152
# model = MobileNetClassifier(img_dims, class_type)
# model = InceptionNetV3Classifier(img_dims, class_type, truncation=False)

model.summary()

# Baseline Segmentation Models

In [ ]:
model = unet(img_dims, 32) # parameter 16, 32
# model = UNetPP(img_dims, 2) # parameter 1, 2
# model = PSPNet(img_dims)
# model = ResUNet(img_dims)
# model= ResUnetPlusPlus(img_dims)
# model = inceptionUnet(img_dims)
# model = DCUNet(img_dims, img_dims, channels=3)
# model = MSRF(img_dims)
# model = swinUNet(img_dims)

model.summary()

# S2C-DeLeNet

## Helper Functions

In [ ]:
def layer_names(model):
    for i, layer in enumerate(model.layers):
        if layer._name == 'top_activation' or layer._name == 'relu' : 
            print(True)
            layer._name = 'convm_b'
        elif layer._name == 'pool4_conv' or layer._name == 'block6a_expand_activation' : 
            print(True)
            layer._name = 'conv5_b'
        elif layer._name == 'pool3_conv' or layer._name == 'block4a_expand_activation' : 
            print(True)
            layer._name = 'conv4_b'
        elif layer._name == 'pool2_conv' or layer._name == 'block3a_expand_activation' : 
            print(True)
            layer._name = 'conv3_b'
        elif layer._name == 'conv1/relu' or layer._name == 'block2a_expand_activation' : 
            print(True)
            layer._name = 'conv2_b'


def decoder_0(convm, conv5, conv4, conv3, conv2, conv1, start_neurons=16, task_='mask'):
    if backbone_variant>0:
        deconv5 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same", name='deconv5_c')(convm)
        uconv5 = concatenate([deconv5, conv5])
        uconv5 = Dropout(0.5)(uconv5)
        uconv5 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same", name='deconv5_b')(uconv5)
        uconv5 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same", name='deconv5_a')(uconv5) 
        convm_ = uconv5
    else:
        convm_=conv5
    
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same", name='deconv4_c')(convm_)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same", name='deconv4_b')(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same", name='deconv4_a')(uconv4)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same", name='deconv3_c')(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same", name='deconv3_b')(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same", name='deconv3_a')(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same", name='deconv2_c')(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same", name='deconv2_b')(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same", name='deconv2_a')(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same", name='deconv1_c')(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same", name='deconv1_b')(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same", name='deconv1_a')(uconv1)
    
    if task_=='mask':
        print('you chose to segment images')
        output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid", name='final_mask')(uconv1)
    elif task_=='recon':
        print('you chose to reconstruct images')
        output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    return output_layer


def build__model(summary_=False, task_='mask'):
    if backbone_variant==-99:
        print('YOU CHOSE UNET+MODIFIED DECODER 1')
        input_layer = Input((img_dims, img_dims, 3))
        # input_layer = tf.image.rgb_to_hsv(input_layer)
        input_ = input_layer

        conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same", name='conv1_a')(input_layer)
        conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same", name='conv1_b')(conv1)
        x1 = conv1
        pool1 = MaxPooling2D((2, 2), name='pool1')(conv1)
        pool1 = Dropout(0.25)(pool1)

        conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same", name='conv2_a')(pool1)
        conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same", name='conv2_b')(conv2)
        x2 = conv2
        pool2 = MaxPooling2D((2, 2))(conv2)
        pool2 = Dropout(0.5)(pool2)

        conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same", name='conv3_a')(pool2)
        conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same", name='conv3_b')(conv3)
        x3 = conv3
        pool3 = MaxPooling2D((2, 2))(conv3)
        pool3 = Dropout(0.5)(pool3)

        conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same", name='conv4_a')(pool3)
        conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same", name='conv4_b')(conv4)
        x4 = conv4
        pool4 = MaxPooling2D((2, 2))(conv4)
        pool4 = Dropout(0.5)(pool4)

        # Middle
        conv5 = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
        conv5 = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same", name='conv5_b')(conv5)
        x5 = conv5
        xm = None


      # out = decoder_1(None, conv5, conv4, conv3, conv2, conv1, start_neurons=start_neurons)
      # model = Model(input_layer, out)

    elif np.abs(backbone_variant)==1:
        model = tf.keras.applications.InceptionV3(False, input_shape=(img_dims, img_dims, 3) )
        input_ = model.input
        if backbone_variant<0:
            x4 = GlobalAveragePooling2D()(model.layers[-83].output) 


    elif np.abs(backbone_variant)>=12 and np.abs(backbone_variant)<20:
        print("You Chose EFFNET")

        if np.abs(backbone_variant)==12: model = tf.keras.applications.EfficientNetB2(False, input_shape=(img_dims, img_dims, 3)  )
        elif np.abs(backbone_variant)==13: model = tf.keras.applications.EfficientNetB3(False, input_shape=(img_dims, img_dims, 3)  )
        elif np.abs(backbone_variant)==14: model = tf.keras.applications.EfficientNetB4(False, input_shape=(img_dims, img_dims, 3)  )

        input_ = model.input

        if backbone_variant>0:
            xm = model.get_layer('top_activation').output #'convm_b'
        else:
            xm= None
        x5 = model.get_layer('block6a_expand_activation').output #'conv5_b'
        x4 = model.get_layer('block4a_expand_activation').output #'conv4_b'
        x3 = model.get_layer('block3a_expand_activation').output #'conv3_b'
        x2 = model.get_layer('block2a_expand_activation').output #'conv2_b'
        x1 = Conv2D(start_neurons , (3, 3), activation="relu", padding="same")(model.input)
        x1 = Conv2D(start_neurons , (3, 3), activation="relu", padding="same", name='conv1_b')(x1)



    elif np.abs(backbone_variant)>=21 and np.abs(backbone_variant)<30:
        print("You Chose DENSENET")
        if np.abs(backbone_variant)==21: model = tf.keras.applications.DenseNet121(False, input_shape=(img_dims, img_dims, 3)  )
        elif np.abs(backbone_variant)==22: model = tf.keras.applications.DenseNet169(False, input_shape=(img_dims, img_dims, 3)  )
        elif np.abs(backbone_variant)==23: model = tf.keras.applications.DenseNet201(False, input_shape=(img_dims, img_dims, 3)  )

        input_ = model.input

        if backbone_variant>0:
            xm = model.get_layer('relu').output #'convm_b'
        else:
            xm= None
        x5 = model.get_layer('pool4_conv').output #'conv5_b'
        x4 = model.get_layer('pool3_conv').output #'conv4_b'
        x3 = model.get_layer('pool2_conv').output #'conv3_b'
        x2 = model.get_layer('conv1/relu').output #'conv2_b'
        x1 = Conv2D(start_neurons , (3, 3), activation="relu", padding="same")(model.input)
        x1 = Conv2D(start_neurons , (3, 3), activation="relu", padding="same", name='conv1_b')(x1)


    if decoder_variant==0:
        out = decoder_0(xm, x5, x4, x3, x2, x1, start_neurons, task_=task_)
    elif decoder_variant==1:
        out = decoder_1(xm, x5, x4, x3, x2, x1, start_neurons)


    model = Model(input_, out)

  # print(x1.shape, x2.shape, x3.shape, x4.shape, x5.shape, xm)

    layer_names(model)
    
    
      
    model.compile(loss= dice_coef_loss, #'binary_crossentropy'
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr_,
                                                      beta_1=0.9,
                                                      beta_2=0.999,
                                                      epsilon=None,
                                                      amsgrad=False),
                metrics=['accuracy',precision, recall, dice_coef, iou, mcc, XOR_Error])

    if summary_: 
        model.summary()
    return model, 2

## Build Custom Segmentation Model

In [ ]:
backbone_variant = 14
                        #  type number mapping ...  
                        #                            1: inception_untr,   -1:inception_tr
                        #                           12: EffB2_untr,      -12: EffB2_tr ,     13: EffB3_untr,      -13: EffB3_tr,     14: EffB4_untr,      -14: EffB4_tr 
                        #                           21: dense121_untr,   -21: dense121_tr,   22: dense169_untr,   -22: dense169_tr,  23: dense201_untr,   -23: dense201_tr

decoder_variant = 0     #  type number mapping ...  0: vanilla,           

start_neurons = 16      #  decoder size: either 16 (light) or 32 (heavy)


print('BV :', backbone_variant, '     DV :', decoder_variant, '    image level neurons :', start_neurons)
print('  ')

model,_ = build__model(True,'mask')

## Build Combined Classification Model

### Build Segmentation Model First

In [ ]:
what_sort = 2           

backbone_variant = 14
                        #  type number mapping ... 
                        #                            1: inception_untr,   -1:inception_tr
                        #                           12: EffB2_untr,      -12: EffB2_tr ,     13: EffB3_untr,      -13: EffB3_tr,     14: EffB4_untr,      -14: EffB4_tr 
                        #                           21: dense121_untr,   -21: dense121_tr,   22: dense169_untr,   -22: dense169_tr,  23: dense201_untr,   -23: dense201_tr

decoder_variant = 0     #  type number mapping ...  0: vanilla,            

start_neurons = 16      #  decoder size: either 16 (light) or 32 (heavy)

print('BV :', backbone_variant, '     DV :', decoder_variant, '    image level neurons :', start_neurons)
print('  ')

test_dict = {"2": build__model}

model,T = test_dict[str(what_sort)](False,task_)


### Load Segmentor Weights

In [ ]:
set_weights = '../Weights/HAM_BV14_DV0_light.h5'

# model.load_weights(set_weights)

for L in model.layers[:]:
    L.trainable = True

### Extract Intermediate Layers

In [ ]:
LOI = ['conv1_', 'conv2_', 'conv3_', 'conv4_', 'conv5_', 'convm_']  #LOI:layer of interest :encoder
LOI2 = ['deconv1_', 'deconv2_', 'deconv3_', 'deconv4_', 'deconv5_', 'convm_']  #LOI:layer of interest : decoder

LOI_name = 'b' #'a''b'

layer_outs = []
for i in range(len(LOI)-(backbone_variant<0)*1):
    if i+1==len(LOI)-(backbone_variant<0)*1:
        LOI[i] = LOI[i]+'b'
    else:
        LOI[i] = LOI[i]+LOI_name
    layer_outs.append(model.get_layer(LOI[i]).output)
    print(layer_outs[-1])

LOI = LOI2
LOI_name = 'a'
for i in range(len(LOI)-(backbone_variant<0)*1):
    if i+1==len(LOI)-(backbone_variant<0)*1:
        LOI[i] = LOI[i]+'b'
    else:
        LOI[i] = LOI[i]+LOI_name
    layer_outs.append(model.get_layer(LOI[i]).output)
    print(layer_outs[-1])

class_segmodel = Model(model.input, [layer_outs[:]])
len(layer_outs[:])


### Build Final Classifier

In [ ]:
def combined_classifier(neurons=16, MLP_parted=1, lighter_=False, classes_=7):   #classify 3 with both encoder and decoder
    X0 = Input((img_dims, img_dims, 3))
    lois = class_segmodel(X0)
    lois1 = lois[0][:6]
    lois2 = lois[0][-6:]
    print(lois[0][6])
    lighter_ = lighter_*1
  
    def feature_propagation(x, z, f):  
        # FCM
        x = Conv2D(f, 3, padding='same', activation=LeakyReLU(0.1))(x)  
        x_ = Conv2D(f, 3, padding='same', activation=LeakyReLU(0.1))(x)
        
        p = DepthwiseConv2D(1, padding='same', activation=LeakyReLU(0.1))(z) 
        p_ = Conv2D(f, 3, padding='same', activation=LeakyReLU(0.1))(p)
        
        x = MaxPool2D((2,2))(x_)
        p = MaxPool2D((2,2))(p_)      

        # 3DLR
        rx = UpSampling2D()(x)
        rp = UpSampling2D()(p)

        p = concatenate([x,p])

        rx = DepthwiseConv2D(1, padding='same', activation=LeakyReLU(0.1))(rx) 
        rx = Conv2D(f, 3, padding='same', activation=LeakyReLU(0.1))(rx)

        rp = DepthwiseConv2D(1, padding='same', activation=LeakyReLU(0.1))(rp) 
        rp = Conv2D(f, 3, padding='same', activation=LeakyReLU(0.1))(rp)
        
        rx = subtract([x_,rx])
        rp = subtract([p_,rp])
        r = concatenate([rx,rp])

        p = Conv2D(2*f, 3, padding='same', activation=LeakyReLU(0.1))(p)
        return p, r

    for k in range(2):
        if k==0:  
          lois_ = lois1
          X=X0
        else:     
          lois_ = lois2
          X=X0

        for i in range(len(lois_)):      
          if i==len(lois_)-1:
            _, tr = feature_propagation(X, lois_[i], neurons*2**(i-k*lighter_))
          else:
            tp, tr = feature_propagation(X, lois_[i], neurons*2**(i-k*lighter_))

          X = tp

        x = GlobalAveragePooling2D()(tp)
        r = GlobalAveragePooling2D()(tr)  

        x = Dense((2**(int(np.log2(x.shape[-1]))))//2, LeakyReLU(0.1))(x)
        x = Dense(x.shape[-1]//4, LeakyReLU(0.1))(x)
        r = Dense((2**(int(np.log2(r.shape[-1]))))//2, LeakyReLU(0.1))(r)
        r = Dense(r.shape[-1]//4, LeakyReLU(0.1))(r)
        x = concatenate([x,r])    
        
        if k==0:          
          x1 = Dense(x.shape[-1], LeakyReLU(0.1))(x)
        else:
          x2 = Dense(x.shape[-1], LeakyReLU(0.1))(x)

    x = concatenate([x1,x2])         
        
    if class_type=='binary':
      x = Dense(1, 'sigmoid')(x)
    else:
      x = Dense(classes_, 'softmax')(x)
    print(x)
    
    model = Model(X0, x)

    return model

### Declare Model

In [ ]:
model = combined_classifier(lighter_=True,classes_=7)

# Data Generator

## For Classification

In [ ]:
dir = '../../SkinData/HAM10000/Split_class/Train' 
dir_val = '../../SkinData/HAM10000/Split_class/Test' 


datagen=ImageDataGenerator(rescale=1./255,
                            rotation_range=15,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            # vertical_flip=True,
                            fill_mode='nearest')
                            
train_gen = datagen.flow_from_directory(directory=dir,
                                        target_size=(img_dims, img_dims), 
                                        batch_size=batch_,
                                        class_mode=class_type,
                                        shuffle=True,)

datagen_test=ImageDataGenerator(rescale=1./255)

val_gen = datagen_test.flow_from_directory(directory=dir_val,
                                    target_size=(img_dims, img_dims),
                                    batch_size=1,
                                    class_mode=class_type,
                                    shuffle=True,)


## For Segmentation

In [ ]:
source_path = '../../SkinData/HAM10000/Split_seg/Train'
source_path_val = '../../SkinData/HAM10000/Split_seg/Val'


def adjust_data(img,mask):
    # img = rgb_cancellation(img)#.........augmentations, if need be
    img = img / 255.
    mask = mask / 255.
    if not task_=='recon':
        mask[mask > 0.5] = 1.0
        mask[mask <= 0.5] = 0.0    
    return (img, mask)

folders = ['Img','Mask']
folders_val=['Img', 'Mask']

def my_generator(batch_size_):
    imggen = ImageDataGenerator(rotation_range=0.1,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest')
    
    data_generator = imggen.flow_from_directory(directory = source_path,
                                                target_size = (img_dims,img_dims),
                                                color_mode = 'rgb',
                                                classes = [folders[0]],
                                                class_mode = None,
                                                batch_size = batch_size_,
                                                seed = seed_,
                                                shuffle = True)
    
    mask_generator = imggen.flow_from_directory(directory = source_path,
                                                target_size = (img_dims,img_dims),
                                                 color_mode = 'grayscale',
                                                 classes = [folders[1]],
                                                class_mode = None,
                                                batch_size = batch_size_,
                                                seed = seed_,
                                                shuffle = True)
    
    train_gen = zip(data_generator, mask_generator)
    
    for (img, mask) in train_gen:
        img, mask = adjust_data(img, mask)
        yield (img,mask)
    

def my_generator_val(batch_size_):

    imggen = ImageDataGenerator()
    
    data_generator = imggen.flow_from_directory(directory = source_path_val,
                                                            target_size = (img_dims,img_dims),
                                                            color_mode = 'rgb',
                                                            classes = [folders_val[0]],
                                                            class_mode = None,
                                                            batch_size = batch_size_,
                                                            seed = seed_,
                                                            shuffle = False)
    mask_generator = imggen.flow_from_directory(directory = source_path_val,
                                                target_size = (img_dims,img_dims),
                                                color_mode = 'grayscale',
                                                classes = [folders_val[1]],
                                                class_mode = None,
                                                batch_size = batch_size_,
                                                seed = seed_,
                                                shuffle = False)

    train_gen = zip(data_generator, mask_generator)
    
    for (img, mask) in train_gen:
        img, mask = adjust_data(img, mask)
        yield (img,mask)

print('Val Img, Label, Train Img, Label')  
len(os.listdir(source_path_val+'/'+folders_val[1])), len(os.listdir(source_path_val+'/'+folders_val[0])), len(os.listdir(source_path+'/'+folders[1])), len(os.listdir(source_path+'/'+folders[0]))

# Training

In [ ]:
log_destination = '../Logs'
weight_destinattion = '../Weights'

run_id = 'ashol_model_mama'   #do this with plot

weight_file = weight_destinattion + '/' + run_id + '.h5'
log_dir = log_destination + '/' + run_id

In [ ]:
if train_segmentation_only==1:
    model.compile(loss= dice_coef_loss, # 'categorical_crossentropy', #'binary_crossentropy' ..  .. weighted_binary_crossentropy .. weighted_binary_crossentropy
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr_),
                metrics=['accuracy',precision, recall, dice_coef, iou, mcc, XOR_Error])

    monitor_ = 'val_dice_coef'
    mode_ = 'max'
    
else:
    model.compile(loss= 'categorical_crossentropy', # 'categorical_crossentropy', #'binary_crossentropy' ..  .. weighted_binary_crossentropy .. weighted_binary_crossentropy
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr_),
                metrics=['accuracy', precision, recall, mcc])

    monitor_ = 'val_accuracy'
    mode_ = 'max'

    
weight_saver = ModelCheckpoint(weight_file, 
                               monitor=monitor_,
                               mode =mode_,
                               save_best_only=True,
                               save_weights_only=True,
                               verbose=1)

annealer = ReduceLROnPlateau(monitor='loss',
                             factor=0.5,
                             patience=5,
                             min_lr=1e-8,
                             verbose=1)

hist_csv_file = weight_destinattion + '/' + run_id+'.csv'
csv_logger = CSVLogger(hist_csv_file, append=True)

factor = batch_

In [ ]:
if train_segmentation_only==1:
      print('*******************ONLY SEGMENTATION*******************')
      ##>>> *** For Segmentation
      history= model.fit(my_generator(batch_),
              steps_per_epoch = len(os.listdir(source_path+'/'+folders[1]))//(factor),
              validation_data = my_generator_val(batch_), 
              validation_steps= len(os.listdir(source_path_val+'/'+folders_val[1]))//(batch_),                          
              epochs=80,
              verbose=1, 
              callbacks = [weight_saver , annealer, csv_logger]
              )
        
elif train_classification_only==1:
      ##>>> *** For Classification
      print('*******************ONLY CLASSIFICATION*******************')
      history= model.fit(train_gen,
                            steps_per_epoch=8019//(batch_),
                            epochs=100,
                            verbose=1,
                            validation_data=val_gen,
                            validation_steps=998//batch_,
                            callbacks=[weight_saver , annealer, csv_logger],
                         )
                         
elif train_combined==1:
    print('*******************BOTH CLASS+SEG*******************')
    ## Combined Training
    history= model.fit(train_gen,
                        steps_per_epoch=8019//(factor),
                        epochs=150,
                        verbose=1,
                        validation_data=val_gen,
                        validation_steps=998, 
                        callbacks = [weight_saver , annealer, csv_logger], 
                     )

else:
    print("Specify which one you want to train")
    

# Evaluation (For Segmentation)

## Quantitative Results

In [ ]:
weight_ = weight_file
model.load_weights(weight_)

source_path_val = '../../SkinData/HAM10000/Split_seg/Val' #'../../SkinData/HAM10000/Split_seg/Test'

model.compile(metrics=['accuracy',precision, recall, dice_coef, iou, mcc, XOR_Error,
                        tf.keras.metrics.AUC(num_thresholds=6, curve='ROC', from_logits=True,
                        thresholds=[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]
                        )
                        ])

if train_segmentation_only:
    evals= model.evaluate(my_generator_val(1), verbose=1, steps=len(os.listdir(source_path_val+'/'+folders_val[1])))

print(weight_)
# print('BV :', backbone_variant, '     DV :', decoder_variant, '    image level neurons :', start_neurons)
print('  ')

evals, source_path_val


## Visual Results

In [ ]:
source_path_val = '../../SkinData/HAM10000/Split_seg/Val' #'../../SkinData/HAM10000/Split_seg/Test'
gt_names = [] 

# weight_ = 'SkinData/Weights/DC-UNet.h5'
weight_ = weight_file
model.load_weights(weight_)

print(source_path_val)
print(weight_)

how_many= 300

X= np.zeros((how_many,img_dims,img_dims,3), dtype=float)
Yh = np.zeros(((how_many, img_dims, img_dims, 1)), dtype=float)
Y = np.zeros(((how_many, img_dims, img_dims)), dtype=float)

instance= 0 %len(os.listdir(source_path_val+'/'+folders_val[1]))

for i in range(how_many):
  print(i)
  rotate_im = 0
  temp = cv2.imread(source_path_val+'/'+folders_val[0]+'/'+np.sort(os.listdir(source_path_val+'/'+folders_val[0]))[instance+i])


  temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
  tempx = cv2.resize(temp, (img_dims,img_dims))
  X[i] = np.rot90(tempx,rotate_im//90,)

  def power_law(x , t, rgb=False):
    x = x**t
    if rgb: 
      x = (x*255).astype(int)  
    return x

  a = 0.5 #random.random()
  contrast = a+0.5
  # print('contrast:', contrast)

  Yh[i] = model.predict(power_law(X[i:i+1]/255.0, contrast))

  temp = cv2.imread(source_path_val+'/'+folders_val[1]+'/'+np.sort(os.listdir(source_path_val+'/'+folders_val[1]))[instance+i])
  
  gt_names.append(np.sort(os.listdir(source_path_val+'/'+folders_val[1]))[instance+i])
  Y[i] = np.rot90(cv2.resize(temp, (img_dims,img_dims))[:,:,0], rotate_im//90,)
  temp2 = (Y[i]>128)*1
  temp2 = np.sum(temp2)/(temp2.shape[0]*temp2.shape[1])
  # print(temp2*100, '% -->', (img_dims**2)*temp2)


np.unique(Yh), np.mean(Yh), (np.max(Yh[0,:,:,0])+np.min(Yh[0,:,:,0]))*0.5


## ROIs

In [ ]:
def colour_code_pred(x,y):
  y = np.expand_dims(y,-1)
  x = np.expand_dims(x,-1)
  y = np.concatenate((y,y,y),-1)
  x[x>=20]=255
  x[x<20]=0


  def colourize(x,r,g,b):
    x[:,:,0][x[:,:,0]>0.1]=r
    x[:,:,1][x[:,:,1]>0.1]=g
    x[:,:,2][x[:,:,2]>0.1]=b
    return x.astype(int)

  tp = (x/255)*y
  fp = (1.0-(x/255))*y
  fn = (x/255)*(1.0-y)

  tp = colourize(tp,0,255,0) #green
  fn = colourize(fn,255,0,0) #red
  fp = colourize(fp,0,80,255) #blue

  return x,y, fp+tp+fn

d_ = 0
p_ = 0
r_ = 0
j_ = 0
f2_= 0
c = 1e-8

show_how= 300
plot_it= True
selections = [173, 164, 88, 299, 263, 215]
the_dices = []
the_dices_idx = []

for i in range(how_many):
  
  Yh_ = 1.0*(Yh[i][:,:,0]>1/2)
  A,B,C= colour_code_pred(Y[i]*1,Yh_)
    
  
  Yh_thresh = 1.0*(Yh[i,:,:,0])

  dice = np.array(dice_coef(Y[i,:,:]/255, Yh_thresh))
  prec = np.array(precision(Y[i,:,:]/255, Yh_thresh))
  dice_range = (dice>=0.60) and (dice<=0.998)

  ALL = (i+1) in selections
  # ALL = True

    
   
  if ALL and (i<show_how and dice_range):
    c+=1
    d_ = d_+np.array(dice_coef(Y[i,:,:]/255, Yh_thresh))
    p_ = p_+ np.array(precision(Y[i,:,:]/255, Yh_thresh))
    r_ = r_+ np.array(recall(Y[i,:,:]/255, Yh_thresh))
    j_ = j_+np.array(iou(Y[i,:,:]/255, Yh_thresh))
    if plot_it:
      plt.figure(figsize=(20,20))
      plt.subplot(1,4,1), plt.imshow(power_law(X[i]/255, contrast , True))
      plt.axis('off')
      plt.title('Main Image')
      plt.subplot(1,4,2), plt.imshow(Y[i],cmap='gray')
      plt.axis('off')
      plt.title('Ground Truth')
      plt.subplot(1,4,3), plt.imshow(Yh[i][:,:,0],cmap='gray')
      plt.axis('off')
      plt.title('Segmented Lesion')
      plt.subplot(1,4,4), plt.imshow(C,cmap='gray')
      plt.axis('off')
      plt.title('Regions of Interests')
      plt.show()

    print(i+1)
    print(gt_names[i])
    print('Current Image model___dice:', np.array(dice_coef(Y[i,:,:]/255, Yh_thresh)),
           'precision:', np.array(precision(Y[i,:,:]/255, Yh_thresh)),
           'recall:', np.array(recall(Y[i,:,:]/255, Yh_thresh)) ,
           'iou:', np.array(iou(Y[i,:,:]/255, Yh_thresh)))
    print('mse:' , np.mean(((Y[i,:,:]/255)-Yh_thresh)**2))
    the_dices.append(np.array(dice_coef(Y[i,:,:]/255, Yh_thresh)))
    the_dices_idx.append(i+1)
    print(c, 'Cummulative model___precision:',p_/(c),' - recall:', r_/(c), '- dice: ', d_/(c),' - iou:', j_/(c))
    

# Evaluation (For Classification)

In [ ]:
data_dir = dir_val #validation/test data  
weight_file = '../Weights/best.h5'
model.load_weights(weight_file)

## Confusion Matrix

In [ ]:
# dir_val="/content/drive/MyDrive/Terms/Thesis/WCE DATA/Binary And Multiple Class/Validation/Val_Normal_Angiectasia"

files_list= np.sort(os.listdir(data_dir))
# model.evaluate(val_gen)

tot=0
tot_classes = len(files_list)
conf_array=np.zeros((tot_classes,tot_classes), dtype=int)

for k in range(tot_classes):
  main_path = data_dir+'/'+files_list[k]  
  for i in range(len(os.listdir(main_path))):
    im = cv2.imread(main_path+'/'+np.sort(os.listdir(main_path))[i])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = resize(im, (img_dims, img_dims, 3))
    im = np.reshape(im, (1, 256, 256,3))
    if np.max(np.unique(im))>1.0:
      im = im/255.0

    p = (model.predict(im[0:1]))[0]
    p_rounded = (p==p[np.argmax(p)])*1.0
    tot+=1
    if class_type=='binary':
       conf_array[k,int(p_rounded[0])]+=1
       if not (i%10): print('class: ', k, ' name: ', files_list[k],'; img ', i, ' of ' ,len(os.listdir(main_path)),'; prediction =', p)
    else:
      conf_array[k,np.argmax(p_rounded)]+=1
      if not (i%10): print('class: ', k, ' name: ', files_list[k],'; img ', i, ' of ' ,len(os.listdir(main_path)),'; prediction =', p_rounded, ' (',tot,')' )
    
    # plt.imshow(im[0])
    # if i>20: break
  # break
print('********************')
# conf_array

print(files_list)
print('\n')
conf_array

## Micro Precision=Recall=F1=Acc


Relavant sites for implementing micro, macro and weighted quantities

 https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1
 
 https://androidkt.com/micro-macro-averages-for-imbalance-multiclass-classification/


In [ ]:
tp = 0
f = 0 


for i in range(conf_array.shape[0]):
  for j in range(conf_array.shape[1]):
      if i==j:  
        tp = tp +conf_array[i,j] 

print('Micro Avg & Accuracy:', tp/(np.sum(conf_array)))

## Macro Precision, Recall, F1

In [ ]:
tp = []
fn = [] 
fp = []

precs = []
recs = []

for i in range(conf_array.shape[0]):
  for j in range(conf_array.shape[1]):
      if i==j:  
        tp.append(conf_array[i,j])
      else:
        fn.append(conf_array[i,j]) 
        fp.append(conf_array[j,i]) 
  recs.append(np.sum(tp)/(np.sum(tp)+np.sum(fn)))
  precs.append(np.sum(tp)/(np.sum(tp)+np.sum(fp)))

  # print(tp, fn, fp)
  # print(recs , precs, '__')
  tp=[]
  fn=[]
  fp=[]
  
PREC = np.mean(precs)
REC = np.mean(recs)

print("Macro Precision:", PREC)
print("Macro Recall:", REC)
print("Macro F1-Score:", 2*PREC*REC/(PREC+REC))

## Weighted Precision, Recall, F1

In [ ]:
label_counts = np.sum(conf_array,axis=1)

weighted_PREC = np.sum([precs[i]*label_counts[i] for i in range(len(precs))])/np.sum(label_counts)
weighted_REC = np.sum([recs[i]*label_counts[i] for i in range(len(recs))])/np.sum(label_counts)

print("Weighted Precision:", weighted_PREC)
print("Weighted Recall:", weighted_REC)
print("Weighted F1-Score:", 2*weighted_PREC*weighted_REC/(weighted_PREC+weighted_REC))